In [1]:
data_var = '2024-02-16'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11881,2024-02-16,Brasil Nbb,19:00,Paulistano,Pinheiros,1.30,3.31,152.5,1.87,1.83,-8.5,2.01,1.72,hzmfFxj2,0.769231,0.302115,0.534759,0.546448,0.071346,0.8,0.2,NaN,NaN,143.634,28.914225,0.201305,174.256,76.819668,0.440844,116.482,167.888,161.88,241.63,0.0,0.0,0.0,0.0,0.616609,0.015289,0.109952,-2.36,-0.472,-0.635593,0.632809,0.5,-0.132809,1.72,0.344,6.715116,0.457435,0.5,0.042565
11882,2024-02-16,Brasil Nbb,19:30,São José,Caxias do Sul,1.23,3.86,156.5,1.85,1.85,-10.5,1.97,1.73,fuRxvMS2,0.813008,0.259067,0.540541,0.540541,0.072075,0.4,0.8,NaN,NaN,157.366,44.324203,0.281663,208.740,124.173227,0.594870,119.282,234.108,135.42,159.10,0.0,0.0,0.0,0.0,0.730723,0.000000,0.091733,-2.37,-0.474,-0.485232,0.615209,0.6,-0.015209,-2.80,-0.560,-5.107143,0.299264,0.2,-0.099264
11883,2024-02-16,Brasil Nbb,19:30,Unifacisa,Botafogo,1.13,5.30,167.5,1.81,1.85,-13.5,2.05,1.68,8rVtwtr9,0.884956,0.188679,0.552486,0.540541,0.073635,0.2,0.6,NaN,NaN,124.342,46.808139,0.376447,318.016,160.416021,0.504428,113.434,318.968,103.50,298.50,0.0,0.0,0.0,0.0,0.917149,0.015456,0.140284,-0.98,-0.196,-0.663265,0.735332,0.8,0.064668,-1.63,-0.326,-13.190184,0.264134,0.2,-0.064134
11884,2024-02-16,Brasil Nbb,20:00,Mogi,União Corinthians,1.62,2.20,150.5,1.80,1.90,-4.5,2.01,1.70,25Upx0cF,0.617284,0.454545,0.555556,0.526316,0.071829,0.4,0.8,NaN,NaN,279.302,157.473880,0.563812,305.848,232.145073,0.759021,215.710,392.156,133.65,342.28,0.0,0.0,0.0,0.0,0.214724,0.038222,0.118169,-3.35,-0.670,-0.925373,0.393783,0.2,-0.193783,-2.47,-0.494,-2.429150,0.242779,0.2,-0.042779
11885,2024-02-16,Eua Nba,00:00,Portland Trail Blazers,Minnesota Timberwolves,4.79,1.21,214.5,1.85,2.09,9.5,2.05,1.89,CWVnE6LK,0.208768,0.826446,0.540541,0.478469,0.035215,0.6,0.6,NaN,NaN,409.378,188.244571,0.459831,212.704,73.261459,0.344429,392.862,230.480,378.23,162.14,1.0,0.0,0.0,0.0,0.843814,0.086145,0.057430,3.97,0.794,4.773300,0.379334,0.5,0.120666,3.19,0.638,0.329154,0.661584,0.8,0.138416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11973,2024-02-16,Mundo Amistoso Internacional,13:30,Bahrain,Jordânia,2.29,1.52,141.5,1.80,1.86,2.5,2.02,1.68,zLmqHD1s,0.436681,0.657895,0.555556,0.537634,0.094576,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.285812,0.023184,0.129955,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11974,2024-02-16,Noruega Blno,15:00,Asker,Gimle,7.50,1.07,164.5,1.84,1.86,14.5,2.02,1.71,UB0xg8SO,0.133333,0.934579,0.543478,0.537634,0.067913,0.4,0.4,NaN,NaN,333.182,140.700107,0.422292,99.212,9.165477,0.092383,170.352,130.094,154.56,104.03,0.0,0.0,0.0,0.0,1.061073,0.007644,0.117535,-2.32,-0.464,-14.008621,0.000000,0.0,0.000000,-1.31,-0.262,-0.267176,0.000000,0.0,0.000000
11975,2024-02-16,Polônia Copa Da Polônia,14:00,Anwil Wloclawek,Torun,1.17,4.83,162.5,1.87,1.87,-12.5,2.06,1.68,8xlq3x5E,0.854701,0.207039,0.534759,0.534759,0.061740,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.862670,0.000000,0.143690,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11976,2024-02-16,Polônia Copa Da Polônia,16:30,Ostrow Wielkopolski,Czarni Slupsk,1.27,3.62,160.5,1.87,1.87,-9.5,2.02,1.71,j1wl2dKK,0.787402,0.276243,0.534759,0.534759,0.063645,0.0,0.0,NaN,NaN,NaN,NaN,

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Portland Trail Blazers,Minnesota Timberwolves,214.5,1.85,1.0000,Over
1,13:00,Croácia Copa Da Croácia,Dubrava,Cibona,158.5,1.85,0.9978,Over
